# Train some models on the features extracted from InceptionV3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.applications.inception_v3 import InceptionV3
#from keras.applications.xception import Xception
from keras.preprocessing import image
#from keras.models import Model
#from keras.layers import GlobalAveragePooling2D

# #Default InceptionV3 options:
# #https://keras.io/applications/#inceptionv3
#
# For pre-trained weights, input_shape has to be  (299, 299, 3) (with 'channels_last' data format)
# or (3, 299, 299) (with 'channels_first' data format).
#
# Note that image.load_img() uses 'channels_last' by default, and you need to
# specify target_size = (299,299) when loading an image to feed into InceptionV3.
#
# keras.applications.inception_v3.InceptionV3(include_top=True
#                                             , weights='imagenet'
#                                             , input_tensor=None
#                                             , input_shape=None
#                                             , pooling=None
#                                             , classes=1000)

import os
from PIL import Image
#Hash images using the ImageHash library
#https://pypi.python.org/pypi/ImageHash
import imagehash

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [188]:
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

## Load the data and pull out features and labels

In [6]:
tree_features_df = pd.read_csv('data/tree_images.csv', '|', index_col=0)
tree_features_df.head()

,p_hash,filename,species,tags,incv3_out_0,incv3_out_1,incv3_out_2,incv3_out_3,incv3_out_4,incv3_out_5,...,incv3_out_2038,incv3_out_2039,incv3_out_2040,incv3_out_2041,incv3_out_2042,incv3_out_2043,incv3_out_2044,incv3_out_2045,incv3_out_2046,incv3_out_2047
0,99a7465a1e99acb4,image_picea_sitchensis_27.png,picea_sitchensis,NaN,13.995308,1.316715,0.279829,67.839417,43.733177,38.206165,...,160.387985,1.176492,45.983730,10.465877,22.118896,3.226861,29.401524,0.486684,0.000000,15.634432
1,93d12f8e712ef068,image_picea_sitchensis_33.png,picea_sitchensis,NaN,7.449574,0.417233,0.447665,6.178374,2.092985,6.251201,...,2.649559,0.523397,0.392520,7.703988,13.842188,12.907540,1.112756,0.000000,5.436511,1.154409
2,cce3482eb991533b,image_picea_sitchensis_6.png,picea_sitchensis,NaN,0.708871,0.640444,1.305804,9.565918,15.157275,1.787687,...,15.114614,0.000000,0.066235,2.280773,8.082823,1.939496,0.041136,0.000000,0.000000,0.034437
3,c119991c4fcf1cda,image_picea_sitchensis_in_winter_12.png,picea_sitchensis,NaN,7.138457,1.225646,15.940742,42.644821,23.604031,13.818824,...,49.895081,0.000000,7.600242,3.260593,13.135204,12.249097,17.588070,0.000000,0.000000,0.000000
4,e3c3dab100fdc29c,image_sitka_spruce_59.png,picea_sitchensis,NaN,0.030419,2.616714,1.435360,52.145359,5.377345,5.631077,...,13.391465,21.970516,5.217728,8.282942,0.788009,0.591635,35.845409,0.000000,0.000000,2.780651


In [8]:
X = tree_features_df.drop(['p_hash', 'filename', 'species','tags'],axis=1)
X.head()

,incv3_out_0,incv3_out_1,incv3_out_2,incv3_out_3,incv3_out_4,incv3_out_5,incv3_out_6,incv3_out_7,incv3_out_8,incv3_out_9,...,incv3_out_2038,incv3_out_2039,incv3_out_2040,incv3_out_2041,incv3_out_2042,incv3_out_2043,incv3_out_2044,incv3_out_2045,incv3_out_2046,incv3_out_2047
0,13.995308,1.316715,0.279829,67.839417,43.733177,38.206165,0.000000,80.229782,43.556065,17.829075,...,160.387985,1.176492,45.983730,10.465877,22.118896,3.226861,29.401524,0.486684,0.000000,15.634432
1,7.449574,0.417233,0.447665,6.178374,2.092985,6.251201,0.772362,1.633119,2.435113,9.189584,...,2.649559,0.523397,0.392520,7.703988,13.842188,12.907540,1.112756,0.000000,5.436511,1.154409
2,0.708871,0.640444,1.305804,9.565918,15.157275,1.787687,0.532695,0.937607,2.542537,18.555613,...,15.114614,0.000000,0.066235,2.280773,8.082823,1.939496,0.041136,0.000000,0.000000,0.034437
3,7.138457,1.225646,15.940742,42.644821,23.604031,13.818824,1.012441,70.799576,18.526180,16.013214,...,49.895081,0.000000,7.600242,3.260593,13.135204,12.249097,17.588070,0.000000,0.000000,0.000000
4,0.030419,2.616714,1.435360,52.145359,5.377345,5.631077,0.888996,59.129547,13.564379,28.716227,...,13.391465,21.970516,5.217728,8.282942,0.788009,0.591635,35.845409,0.000000,0.000000,2.780651


In [9]:
# y = pd.get_dummies(tree_features_df['species'])
# y.head()

,acer_macrophylum,picea_sitchensis,platanus_acerifolia
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


In [17]:
y = tree_features_df['species']
y.head()

0    picea_sitchensis
1    picea_sitchensis
2    picea_sitchensis
3    picea_sitchensis
4    picea_sitchensis
Name: species, dtype: object

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 0 to 1013
Columns: 2048 entries, incv3_out_0 to incv3_out_2047
dtypes: float64(2048)
memory usage: 15.9 MB


In [19]:
y.describe()

count                 1014
unique                   3
top       acer_macrophylum
freq                   370
Name: species, dtype: object

## Get a train-test split of the data

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=359)

In [21]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 811 entries, 304 to 996
Columns: 2048 entries, incv3_out_0 to incv3_out_2047
dtypes: float64(2048)
memory usage: 12.7 MB


In [22]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203 entries, 264 to 747
Columns: 2048 entries, incv3_out_0 to incv3_out_2047
dtypes: float64(2048)
memory usage: 3.2 MB


## Try logistic regression using one vs. rest (i.e. 3 separate logistic regressions)

Looks like we need lots of regularization since there are so many features

In [90]:
#log_model = LogisticRegression(multi_class='multinomial', solver='saga', C=0.5, max_iter=1000)
log_model = LogisticRegression(multi_class='ovr', solver='liblinear', C=0.00001, max_iter=1000)

In [91]:
log_model.fit(X_train, y_train)

LogisticRegression(C=1e-05, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [92]:
y_test_hat = log_model.predict_proba(X_test)

In [93]:
y_test_hat[:10]

array([[ 0.54860134,  0.22469948,  0.22669918],
       [ 0.44574158,  0.25163863,  0.30261979],
       [ 0.33414079,  0.2790197 ,  0.38683951],
       [ 0.41732966,  0.23454815,  0.34812219],
       [ 0.40272523,  0.33680587,  0.26046889],
       [ 0.2997952 ,  0.38809015,  0.31211465],
       [ 0.47448854,  0.22082997,  0.30468148],
       [ 0.45033394,  0.42345365,  0.12621241],
       [ 0.37153535,  0.56208182,  0.06638283],
       [ 0.45625385,  0.11013843,  0.43360772]])

In [63]:
y_test_true = pd.get_dummies(y_test)
y_test_true.head(10)

,acer_macrophylum,picea_sitchensis,platanus_acerifolia
264,0,1,0
987,1,0,0
782,1,0,0
778,1,0,0
286,0,1,0
481,0,0,1
156,0,1,0
107,0,1,0
19,0,1,0
349,0,0,1


In [94]:
#1.88 with C=0.01
#1.17 with C=0.001
#0.987 with C=0.0001
#0.984 with C=0.00001
log_loss(y_test, y_test_hat)

0.98397553264916515

In [95]:
y_test_pred_cat = log_model.predict(X_test)

In [96]:
#about 0.48 with C=0.001 or C=0.01
#0.49 with C=0.0001
#0.502 with C=0.00001
accuracy_score(y_test, y_test_pred_cat)

0.50246305418719217

## Try logistic regression with multinomial probabilities

Note: The solver doesn't converge if the regularization is too weak.

Looks like we get a slight improvement over the one vs. rest scheme.  Looks like the best C is around 0.00001

In [194]:
log_model_multi = LogisticRegression(multi_class='multinomial', solver='sag', C=0.00001, max_iter=1000)

In [195]:
log_model_multi.fit(X_train, y_train)

LogisticRegression(C=1e-05, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [196]:
y_test_hat_multi = log_model_multi.predict_proba(X_test)

In [197]:
#0.97799 with C=0.00005
#0.9783598 with C=0.00001
#0.9891089 with C=0.000005
#1.021896  with C=0.000001
log_loss(y_test, y_test_hat_multi)

0.97835782118211412

In [198]:
y_test_pred_cat_multi = log_model_multi.predict(X_test)

In [199]:
#0.517 with C=0.00005
#0.522 with C=0.00001
#0.517 with C=0.000005
#0.4926 with C=0.000001
accuracy_score(y_test, y_test_pred_cat_multi)

0.52216748768472909

In [193]:
log_model_multi.classes_

array(['acer_macrophylum', 'picea_sitchensis', 'platanus_acerifolia'], dtype=object)

## Function to wrap a confusion matrix in a dataframe

In [204]:
def confusion_df(y_true, y_pred, sklearn_model):
    return pd.DataFrame(confusion_matrix(y_true, y_pred),
                           index=['Actual {}'.format(label) for label in sklearn_model.classes_],
                           columns=['Predicted {}'.format(label) for label in sklearn_model.classes_])

In [205]:
confusion_df(y_test, y_test_pred_cat_multi, log_model_multi)

,Predicted acer_macrophylum,Predicted picea_sitchensis,Predicted platanus_acerifolia
Actual acer_macrophylum,46,13,18
Actual picea_sitchensis,27,30,11
Actual platanus_acerifolia,20,8,30


## Try a random forest

This isn't doing as well as logistic regression. I haven't tried pre-pruning the trees, though.

In [181]:
rf_model = RandomForestClassifier(n_estimators=150, max_depth=7, oob_score=True, n_jobs=-1, random_state=25)

In [182]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [183]:
#1.0255954030874306 with n_estimators=50 (seed=359)
#1.0078875073836353 with n_estimators=100 (seed=359)
#0.99548093251202763 with n_estimators=125 (seed=90)
#0.99277943264842983 with n_estimators=150, max_depth=10 (seed=90)
#0.99532292044623072 with n_estimators=150, max_depth=7 (seed=25)
#1.0060613930521483 with n_estimators=150 (seed=359)
#1.0066156011904319 with n_estimators=200 (seed=359)
y_test_hat = rf_model.predict_proba(X_test)
log_loss(y_test, y_test_hat)

0.99532292044623072

In [184]:
#0.41871921182266009 with n_estimators=50 (seed=359)
#0.41379310344827586 with n_estimators=100 (seed=359)
#0.45320197044334976 with n_estimators=125 (seed=90) 
#0.47290640394088668 with n_estimators=150, max_depth=10 (seed=90)
#0.48768472906403942 with n_estimators=150, max_depth=7 (seed=25)
#0.45320197044334976 with n_estimators=150 (seed=359)
#0.45320197044334976 with n_estimators=200 (seed=359)
y_test_hat_cat = rf_model.predict(X_test)
accuracy_score(y_test, y_test_hat_cat)

0.48768472906403942

In [185]:
rf_model.oob_score_

0.4722564734895191

## Try a Gradient Booster

This is the best so far, with the right parameters... also the slowest to train

In [172]:
gb_model = GradientBoostingClassifier(learning_rate=0.01, n_estimators=200, subsample=0.5, max_depth=5)

In [173]:
gb_model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=None, subsample=0.5, verbose=0,
              warm_start=False)

In [186]:
#0.9469096393142834 with learning_rate=0.01, n_estimators=200, subsample=0.5, max_depth=5 (Best so far)
#0.98792578185982549 with learning_rate=0.01, n_estimators=200, subsample=0.5, max_depth=3
#1.0309311779727697 with learning_rate=0.1, n_estimators=100, subsample=0.5, max_depth=3
y_test_hat = gb_model.predict_proba(X_test)
log_loss(y_test, y_test_hat)

0.9469096393142834

In [187]:
#0.54679802955665024 with learning_rate=0.01, n_estimators=200, subsample=0.5, max_depth=5 (Best so far)
#0.47290640394088668 with learning_rate=0.01, n_estimators=200, subsample=0.5, max_depth=3
#0.47290640394088668 with learning_rate=0.1, n_estimators=100, subsample=0.5, max_depth=3
y_test_hat_cat = gb_model.predict(X_test)
accuracy_score(y_test, y_test_hat_cat)

0.54679802955665024

In [190]:
gb_model.classes_

array(['acer_macrophylum', 'picea_sitchensis', 'platanus_acerifolia'], dtype=object)

## Let's check out the confusion matrix and misclassifications

In [207]:
confusion_df(y_test, y_test_hat_cat, gb_model)

,Predicted acer_macrophylum,Predicted picea_sitchensis,Predicted platanus_acerifolia
Actual acer_macrophylum,46,17,14
Actual picea_sitchensis,26,30,12
Actual platanus_acerifolia,18,5,35
